In [1]:
import os
from pathlib import Path
import pandas as pd
from obspy.core import read, UTCDateTime, Stream
import obspy
from preprocessing_tools import *

In [2]:
directorio_recortado = "Datos_Trim/"
evid = "2018.08.12.14.42.09" # Sismo mas fuerte
evdir = directorio_recortado + evid
station_file = "station.csv"
catalog = 'catalog.dat'
ev_st = "ev_st.csv"

sacdir = "%s/RT"%evdir
outdir = evdir

P = Path(sacdir)
df_catalog = read_catalog(catalog)
stations_meta = pd.read_csv(station_file)
ev_st_df = pd.read_csv(ev_st)

st = obspy.read(sacdir+'/.*')

In [9]:
ev_row = df_catalog.loc[df_catalog['date_time_id'] == evid]

In [10]:
ev_row

,date_time,lat,lon,depth,mag,corr,date_time_id
8,2018-08-12T14:42:09.000000Z,17.11,-100.84,44.3,5.2,"[0.9316, 51.61, 20180812144209.0]",2018.08.12.14.42.09


In [3]:
# Filter
freqmin = 0.02
freqmax = 0.05
corners = 3

# Desired sampling interval
dt = 1.0

# Reduction velocity in km/sec, 0 sets the reference time to origin time
vred = 0

# time before and after reference time, data will be cut before and after the reference time
time_before = 30
time_after = 200

In [4]:
df_catalog.head(10)

,date_time,lat,lon,depth,mag,corr,date_time_id
0,2018-08-12T01:17:47.060000Z,17.180,-100.950,3.1,1.20,"[0.7309, 37.06, 20180812011747.06]",2018.08.12.01.17.47
1,2018-08-12T01:37:43.000000Z,17.180,-101.020,26.9,3.90,"[1.0, 52.15, 20180812013743.0]",2018.08.12.01.37.43
2,2018-08-12T02:06:47.640000Z,17.235,-101.435,26.0,2.41,"[0.5361, 22.24, 20180605040434.0]",2018.08.12.02.06.47
3,2018-08-12T02:21:18.100000Z,17.030,-100.900,69.7,2.25,"[0.5164, 21.02, 20180810013340.0]",2018.08.12.02.21.18
4,2018-08-12T05:13:06.000000Z,16.490,-100.580,6.4,4.30,"[0.9795, 46.39, 20180812051306.0]",2018.08.12.05.13.06
5,2018-08-12T06:23:37.380000Z,17.190,-101.065,24.0,1.98,"[0.4617, 23.95, 20180514151444.0]",2018.08.12.06.23.37
6,2018-08-12T11:45:10.000000Z,16.950,-100.380,31.9,3.60,"[0.9133, 45.26, 20180812114510.0]",2018.08.12.11.45.10
7,2018-08-12T13:21:54.400000Z,16.970,-100.330,38.1,2.20,"[0.8851, 42.04, 20180812132154.39]",2018.08.12.13.21.54
8,2018-08-12T14:42:09.000000Z,17.110,-100.840,44.3,5.20,"[0.9316, 51.61, 20180812144209.0]",2018.08.12.14.42.09
9,2018-08-12T15:06:37.940000Z,17.200,-100.770,40.3,2.20,"[0.9456, 46.73, 20180812150637.93]",2018.08.12.15.06.37


In [5]:
if vred:
    p = 1/vred
else:
    p = 0
    
st.filter("bandpass",freqmin=freqmin,freqmax=freqmax,corners=corners,zerophase=True)
st.taper(max_percentage=0.05)
origin_time = df_catalog.loc[df_catalog['date_time_id'] == evid]['date_time'].values[0]
print(origin_time)

# Trim and decimate the data
for tr in st:
    tr.decimate(factor=int(tr.stats.sampling_rate*dt), strict_length=False, no_filter=True)
    tr.resample(1/dt, strict_length=False, no_filter=True)
    # Recortar aun mas los datos en base a la distancia entre la estacion y el evento
    tr.stats.sac.t1 = origin_time + p*(tr.stats.sac.dist)
    tr.trim(tr.stats.sac.t1-time_before,tr.stats.sac.t1+time_after,pad=True,fill_value=0)
    tr.data = 100*tr.data # m/s to cm/s
    tr.stats.sac.b = -1*(origin_time - tr.stats.starttime)
    tr.stats.sac.o = 0
    # Save final trace using tdmtpy file name format
    sacout = "%s/%s.%s.dat"%(outdir,tr.id[:-4],tr.id[-1])
    tr.write(sacout,format="SAC")

2018-08-12T14:42:09.000000Z


In [7]:
model = "gil7"
depths = sorted([10,20,round(df_catalog.loc[df_catalog['date_time_id'] == evid]["depth"].values[0])]) # Rango de profundidades para las funciones de Green
npts = int(256) # numero de puntos
t0 = int(0)
ev_st_sub = ev_st_df.loc[ev_st_df['date_time_id'] == evid].copy()
#print(ev_st_sub)

# Directorio de funciones de Green
green_dir = "%s/%s"%(evdir,model)
Path(green_dir).mkdir(parents=True,exist_ok=True)
    
for depth in depths:
    dfile = ("{dist:.0f} {dt:.2f} {npts:d} {t0:d} {vred:.1f}\n")
    dfile_out = "%s/dfile"%evdir
    with open(dfile_out,"w") as f:
        # Distancia con cada estacion
        for i, row in ev_st_sub.iterrows():
            f.write(dfile.format(dist=row.distance,dt=dt,npts=npts,t0=t0,vred=vred))

    # Generate the synthetics
    os.system("hprep96 -M %s.d -d %s -HS %.4f -HR 0 -EQEX"%(model,dfile_out,depth))
    !hspec96
    !hpulse96 -D -i > file96
    !f96tosac -B file96

    greens = ("ZDD","RDD","ZDS","RDS","TDS","ZSS","RSS","TSS","ZEX","REX")

    for i,row in stations_meta.iterrows():      
        for j,grn in enumerate(greens):
            sacin = "B%03d%02d%s.sac"%(i+1,j+1,grn)
            sacout = "%s/%s.%s.%.4f"%(green_dir,row.kstnm,row.kcmpnm,depth)
            tmp = read(sacin,format="SAC")
            tmp.filter('bandpass',freqmin=freqmin,freqmax=freqmax,corners=corners,zerophase=True)
            tmp.write("%s.%s"%(sacout,grn),format="SAC") # overwrite

!\rm B*.sac

  0.9765625E-02  0.1000000E+01
        256         1       129
 ieqex=            0  EARTHQUAKE + EXPLOSION
   10
  TOP  OF MODEL IS FREE SURFACE  
  BASE OF MODEL IS HALFSPACE WITH PROPERTIES OF BOTTOM LAYER
 gil7.d
 LAYER             H      P-VEL     S-VEL   DENSITY  
     1          1.000     3.200     1.500     2.280
     2          2.000     4.500     1.400     2.280
     3          1.000     4.800     2.780     2.580
     4          1.000     5.510     3.180     2.580
     5         12.000     6.210     3.400     2.680
     6          8.000     6.890     3.980     3.000
     7          0.000     7.830     4.520     3.260
 Working model
        d         a         b         rho      1/qa      1/qb       bsh    1/qbsh
      1.000     3.200     1.500     2.280  0.001667  0.003333     1.500     2.280
      2.000     4.500     1.400     2.280  0.001667  0.003333     1.400     2.280
      1.000     4.800     2.780     2.580  0.001667  0.003333     2.780     2.580
      1.000     5.510 

In [13]:
headers = dict(datetime=ev_row["date_time"],
               longitude=ev_row["lon"],
               latitude=ev_row["lat"],
               depth=",".join([ "%.4f"%d for d in depths]),
               path_to_data=evdir,
               path_to_green=green_dir,
               green="herrmann",
               components="URT",
               degree=5,
               weight="dihprep96stance",
               plot=1,
               correlate=0,
              )

pd.options.display.float_format = "{:,.2f}".format
frame = {"station": stations_meta["kstnm"]}
df_out = pd.DataFrame(frame)
df_out[["distance","azimuth"]] = ev_st_df[["distance","azimuth"]]
df_out["ts"] = int(30)
df_out["npts"] = int(150)
df_out["dt"] = dt
df_out["used"] = 1
df_out[["longitude","latitude"]] = stations_meta[["stlo","stla"]]

In [14]:
with open("mtinv.in","w") as f:
    for key, value in headers.items():
        f.write("{0:<15}{1}\n".format(key,value))
    f.write(df_out.to_string(index=False))
    
!cat mtinv.in

datetime       8    2018-08-12T14:42:09.000000Z
Name: date_time, dtype: object
longitude      8   -100.84
Name: lon, dtype: float64
latitude       8   17.11
Name: lat, dtype: float64
depth          10.0000,20.0000,44.0000
path_to_data   Datos_Trim/2018.08.12.14.42.09
path_to_green  Datos_Trim/2018.08.12.14.42.09/gil7
green          herrmann
components     URT
degree         5
weight         dihprep96stance
plot           1
correlate      0
station  distance  azimuth  ts  npts   dt  used  longitude  latitude
   1S01 59,255.71   157.94  30   150 1.00     1    -100.74     16.68
   1S01 59,255.71   157.94  30   150 1.00     1    -100.74     16.68
   1S01 59,255.71   157.94  30   150 1.00     1    -100.74     16.68
   1S03 67,250.89   128.92  30   150 1.00     1    -100.46     16.80
   1S03 67,250.89   128.92  30   150 1.00     1    -100.46     16.80
   1S03 67,250.89   128.92  30   150 1.00     1    -100.46     16.80
   1S04 35,002.30   144.14  30   150 1.00     1    -100.76     16.92
   1